In [2]:
%load_ext watermark
%watermark

%load_ext autoreload
%autoreload 2


# import standard libs
from IPython.display import display
from IPython.core.debugger import set_trace as bp
from pathlib import PurePath, Path
import sys
import time
from collections import OrderedDict as od
import re
import os
import json
import datetime
import pickle


# import python scientific stack
import pandas as pd
import pandas_datareader.data as web
pd.set_option('display.max_rows', 10)
from dask import dataframe as dd
from dask.diagnostics import ProgressBar
from multiprocessing import cpu_count
pbar = ProgressBar()
pbar.register()
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from numba import jit
import math
# import ffn


# import visual tools
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-talk')
plt.style.use('bmh')
#plt.rcParams['font.family'] = 'DejaVu Sans Mono'
plt.rcParams['font.size'] = 9.5
plt.rcParams['font.weight'] = 'medium'
plt.rcParams['figure.figsize'] = 10,7
blue, green, red, purple, gold, teal = sns.color_palette('colorblind', 6)

RANDOM_STATE = 777

print()

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-09-12T13:10:53.552957-04:00

Python implementation: CPython
Python version       : 3.8.19
IPython version      : 8.12.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



/var/folders/j0/8v9qcjfx15g5ftmsy5n0qrq80000gn/T/ipykernel_63265/2173331131.py:46: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-talk')


In [3]:
import os

# Run the setup script
%run ../../config/setup_project.py

# Call the function to set up the project path
setup_project_path()

# Now you can import your modules
from src.utils import helper as h_
import src.ch_02.code_ch_02 as f_ch2
import src.ch_03.code_ch_03 as f_ch3
import src.ch_04.code_ch_04 as f_ch4
import src.ch_05.code_ch_05 as f_ch5
import src.ch_06.code_ch_06 as f_ch6
import src.ch_07.code_ch_07 as f_ch7
import src.ch_08.code_ch_08 as f_ch8

Project root added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management
Config path added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management/config
Current sys.path: ['/Users/paulkelendji/miniconda3/envs/financial_math/lib/python38.zip', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/lib-dynload', '', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages/setuptools/_vendor', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management/config']
Project root added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management
Config path added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management/config
Current sys.path: ['/Users/paulkelendji/miniconda3/envs/financ

---

Let's break this down step by step:

### 1. **What is ADF (Augmented Dickey-Fuller Test)?**
The ADF test is a statistical test used to check whether a time series is **stationary**. In simple terms, a stationary series has statistical properties like the mean, variance, and autocorrelation that do not change over time.

- **Null Hypothesis**: The ADF test's null hypothesis is that the time series has a unit root, meaning it is **non-stationary**.
- **Alternative Hypothesis**: If the null hypothesis is rejected, it means that the time series is **stationary**.

### 2. **Stationarity and Raw Prices:**
The passage refers to running the ADF test on **raw prices**. In finance, raw prices refer to the actual prices of assets, like stock prices.

- **Stationarity in Prices**: If the ADF test **rejects the null hypothesis**, it suggests that the raw prices are **stationary**—meaning that the prices have a constant variance over time and do not exhibit trends or random walks.

### 3. **Implication for Returns:**
Returns are typically calculated as the percentage change in prices from one period to the next. Mathematically:
\[
\text{return}_{t} = \frac{y_{t}}{y_{t-1}} - 1
\]
If prices are stationary, this has implications for the behavior of returns:
- For stationary prices to exist, the **volatility (variance) of returns must change** depending on the price level. This means that as prices rise or fall, the volatility of returns must adjust accordingly to maintain stationarity in prices.
- In other words, **returns are not time-invariant**; the volatility of returns (their risk) will vary with the price levels to ensure the stationarity of prices.

### 4. **Volatility Adjustment:**
- When prices **rise**, the volatility of returns must **decrease**. This keeps the variance of the prices constant because larger prices would otherwise result in greater fluctuations.
- When prices **fall**, the volatility of returns must **increase** to balance out the shrinking price levels and still maintain a constant price variance.

This means that for the prices to be stationary, the returns must have **heteroscedasticity** (variance that changes over time), as their variance is influenced by price levels.

### 5. **Heteroscedasticity in Returns:**
- **Heteroscedasticity** means that the variance (volatility) of a time series is not constant over time.
- If the volatility of returns does not adjust with price levels, the model would be **structurally heteroscedastic**. This is problematic because it suggests that the assumption of constant variance is violated, and the returns behave inconsistently across different price levels.

### Conclusion:
If you run the ADF test on raw prices and reject the null hypothesis (meaning the prices are stationary), it implies that **the volatility of returns must adjust with price levels** to keep the prices stationary. This implies that returns are **heteroscedastic**—their variance changes with price levels—because a constant variance in returns would not support stationary prices. If you assume constant variance in returns, your model would incorrectly handle the price data, leading to structural issues like **heteroscedasticity**. 

Let me know if this clears up the concept!

---

So if i understood correclty :

if we run an ADF test on the raw prices and reject the null hypothesis, it means\
the prices are stationary with finite variance. This implies that retuns \
$\frac{y_{t}}{y_{t-1}} - 1$ or not time invariant (not heteroscedastic).

However, we know that as prices rise, the volatility of returns decreases, and\
as prices fall, the volatility of returns increases. This means that returns'
variance is variant to price levels. We therefore get the contradiction since\
returns are not time invariant.

---

You're raising a valid point, and let's clarify it more carefully because it’s a nuanced concept.

### Restating the key points:
1. **If you run the ADF test on raw prices and reject the null hypothesis**:
   - This suggests that **prices are stationary**, which means that their variance does not change over time.

2. **Implication for returns**:
   - If prices are stationary, the variance of returns must **change** (i.e., returns are not time-invariant).
   - In a stationary price series, the returns’ volatility (variance) needs to **adjust** according to the price level in order to maintain stationarity. This introduces **heteroscedasticity** (i.e., non-constant variance) in returns.

### Clarifying the apparent contradiction:
1. **Returns are typically heteroscedastic**:
   - In financial markets, returns often exhibit **heteroscedasticity**—their volatility tends to change over time. For example, as prices rise, volatility might decrease, and as prices fall, volatility might increase.

2. **If raw prices are stationary**:
   - If you reject the ADF null hypothesis and conclude that **prices are stationary**, it means that the price series does not trend or have a unit root and has a constant variance.
   - For prices to be stationary, **returns cannot have constant variance**. Instead, their variance must **adjust** dynamically to keep the overall price variance stable. In other words, returns must have **heteroscedasticity**—their volatility must change depending on price levels.

### No contradiction in theory:
- If the returns’ variance were constant, then prices **would not be stationary**. This is because constant variance in returns would lead to price levels that fluctuate too much as prices rise or fall, making the price series non-stationary.
- The **correct conclusion** is that **returns’ variance must be variable (heteroscedastic)** if prices are stationary. Therefore, the variability of returns' volatility **prevents a contradiction** and is required for the prices to maintain stationarity.

### Summary:
- When we reject the null hypothesis of the ADF test on **raw prices**, the implication is that the price series is stationary. 
- For the price series to remain stationary, the variance of returns cannot be constant; it must adjust according to the price levels, meaning that **returns are heteroscedastic**.
- This is not a contradiction but rather a consequence of how volatility and price levels interact in financial markets.

Let me know if this resolves your concern or if further clarification is needed!